<a href="https://colab.research.google.com/github/LRManamperi/Machine-Learning/blob/main/tinyML/PlantLeaves_mirco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####  **Importing Required Libraries**

**Objective**  
This section introduces the libraries and modules required for this lab. These libraries enable efficient data handling, preprocessing, and model building for TinyML applications.

**Key Components**  
- **TensorFlow**: A powerful framework for building and deploying machine learning models, especially suited for deep learning tasks.  
- **TensorFlow Datasets (TFDS)**: A module to download and preprocess datasets like PlantVillage efficiently.  
- **Keras Layers and Models**: Used to customize the architecture for our binary classification task.  
- **Adam Optimizer**: A widely-used optimization algorithm for training neural networks efficiently.


In [ ]:
# Mount Google Drive
# Tip: You may need to grant permission in Colab; rerun if path errors occur.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y keras tensorflow tensorflow-model-optimization
!pip install tensorflow==2.12 tensorflow-model-optimization

Found existing installation: keras 3.10.0
Uninstalling keras-3.10.0:
  Successfully uninstalled keras-3.10.0
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
save_dir = '/content/drive/My Drive/TinyPlants'

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

#### **Preparing the PlantVillage Dataset**

**Objective**  
This section focuses on loading, preprocessing, and splitting the PlantVillage dataset into training, validation, and test sets while converting it into a binary classification dataset.

**Steps**  
1. **Load the Dataset**: Utilize TensorFlow Datasets (TFDS) to download and load the PlantVillage dataset.  
2. **Define Healthy Classes**: Specify the "healthy" classes from the dataset's class list. All other classes are automatically labeled as "unhealthy."  
3. **Relabel the Dataset**: Implement a mapping function to convert multi-class labels into binary labels (`0` for healthy, `1` for unhealthy).  
4. **Split the Dataset**: Divide the dataset into training (40,000 samples), validation (7,000 samples), and test sets.  
5. **Preprocess Images**: Resize images to the input size expected by MobileNet and normalize pixel values to the range [0, 1].

**Outcome**  
This step produces preprocessed datasets ready for training, validation, and evaluation, tailored for binary classification tasks.


In [ ]:
# Load the PlantVillage dataset
dataset, info = tfds.load('plant_village', with_info=True, as_supervised=True)

# Define the "healthy" classes
healthy_classes = [
    'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy',
    'Corn___healthy', 'Grape___healthy', 'Peach___healthy',
    'Pepper,_bell___healthy', 'Potato___healthy', 'Raspberry___healthy',
    'Soybean___healthy', 'Strawberry___healthy', 'Tomato___healthy'
]

# Get class names from the dataset info
class_names = info.features['label'].names

In [ ]:
# Corrected function to map original labels to binary labels
def map_to_binary_label(image, label):
    class_name = tf.gather(class_names, tf.cast(label, tf.int32))  # Convert label to int and get class name
    new_label = tf.cond(
        tf.reduce_any(tf.equal(class_name, healthy_classes)),
        lambda: tf.constant(0),  # Healthy
        lambda: tf.constant(1)   # Unhealthy
    )
    return image, new_label

binary_train_dataset = dataset['train'].map(map_to_binary_label)

In [ ]:
# Load the PlantVillage dataset
dataset, info = tfds.load('plant_village', with_info=True, as_supervised=True)

# Define the "healthy" classes
healthy_classes = [
    'Apple___healthy', 'Blueberry___healthy', 'Cherry___healthy',
    'Corn___healthy', 'Grape___healthy', 'Peach___healthy',
    'Pepper,_bell___healthy', 'Potato___healthy', 'Raspberry___healthy',
    'Soybean___healthy', 'Strawberry___healthy', 'Tomato___healthy'
]

# Get class names from the dataset info
class_names = info.features['label'].names

# Function to map original labels to binary labels
def map_to_binary_label(image, label):
    class_name = class_names[label]
    if class_name in healthy_classes:
        new_label = 0  # Healthy
    else:
        new_label = 1  # Unhealthy
    return image, new_label

# Corrected function to map original labels to binary labels
def map_to_binary_label(image, label):
    class_name = tf.gather(class_names, tf.cast(label, tf.int32))  # Convert label to int and get class name
    new_label = tf.cond(
        tf.reduce_any(tf.equal(class_name, healthy_classes)),
        lambda: tf.constant(0),  # Healthy
        lambda: tf.constant(1)   # Unhealthy
    )
    return image, new_label

# Apply the corrected mapping function
binary_train_dataset = dataset['train'].map(map_to_binary_label)


# Split into training, validation, and test sets
train_dataset = binary_train_dataset.take(40000)
val_dataset = binary_train_dataset.skip(40000).take(7000)
test_dataset = binary_train_dataset.skip(47000)

# Image preprocessing
IMG_SIZE = (64, 64)
BATCH_SIZE = 32

def preprocess_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # Normalize to [0,1] range
    return image, label

# Preprocess the datasets
train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


#### **Visualizing Healthy and Unhealthy Leaves**

**Objective**  
This section introduces a visualization step to better understand the dataset. It plots examples of healthy and unhealthy plant leaves from the training dataset.

**Steps**  
1. **Helper Functions**:  
   - `plot_images`: Displays a grid of images with labels.  
   - `collect_examples`: Extracts a specified number of healthy and unhealthy images for visualization.  
2. **Visualization**: Visualize 5 healthy and 5 unhealthy plant leaves from the training dataset.  
   - Healthy leaves are labeled as `Healthy`.  
   - Unhealthy leaves are labeled as `Unhealthy`.

**Outcome**  
This step helps participants familiarize themselves with the dataset and ensures the binary labeling was applied correctly. It also emphasizes the importance of understanding the dataset before training a model.


In [ ]:
import matplotlib.pyplot as plt

# Helper function to display images with labels
def plot_images(images, labels, title):
    plt.figure(figsize=(10, 5))
    for i in range(len(images)):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i])
        plt.title('Healthy' if labels[i] == 0 else 'Unhealthy')
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

# Function to collect 5 healthy and 5 unhealthy examples
def collect_examples(dataset, num_per_class=5):
    healthy_images = []
    unhealthy_images = []
    for image, label in dataset.unbatch():  # Unbatch to iterate over individual samples
        if len(healthy_images) < num_per_class and label.numpy() == 0:
            healthy_images.append(image.numpy())
        elif len(unhealthy_images) < num_per_class and label.numpy() == 1:
            unhealthy_images.append(image.numpy())
        # Stop once we have enough examples
        if len(healthy_images) == num_per_class and len(unhealthy_images) == num_per_class:
            break
    return healthy_images, unhealthy_images

# Collect examples from the training dataset
healthy_images, unhealthy_images = collect_examples(train_dataset)

# Plot healthy and unhealthy images
plot_images(healthy_images, [0] * len(healthy_images), title="Healthy Plant Leaves")
plot_images(unhealthy_images, [1] * len(unhealthy_images), title="Unhealthy Plant Leaves")


# A Simple CNN with Mixed Precision for Binary Classification

## Objective
This section focuses on building a simple CNN model with mixed precision to classify plant leaves as healthy or unhealthy. Mixed precision allows faster computation and reduced memory usage by leveraging the efficiency of lower-precision arithmetic.



## Steps

1. **Create CNN Model**: Create a simple CNN with few convolusion layers and inut layer matching dimensions for a 64*64 image and output lyaer wiht 2 neurons as thi is a binary classification.

2. **Compile the Model**: Use the Adam optimizer with a learning rate of 0.001, binary cross-entropy loss function, and accuracy as a performance metric.

3. **Optimize Dataset Pipeline**: Use TensorFlow's `tf.data` API to prefetch data, ensuring efficient data loading during training and validation.

4. **Train the Model**: Train the model using the prepared training and validation datasets for 10 epochs. The mixed precision policy improves training speed on supported hardware.

5. **Evaluate the Model**: Test the model on the test dataset to evaluate its performance and display the test accuracy.

6. **Save the Model**: Save the trained CNN model in the recommended `.keras` format for future deployment or inference tasks.

## Outcome
By the end of this section, participants will have:
- Trained a compact, efficient model optimized for TinyML applications.
- Leveraged mixed precision training for faster computation and reduced memory consumption without compromising model accuracy.
- Saved the trained model in a modern format for deployment on resource-constrained devices.




In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Use float32 for clean INT8 PTQ conversion
tf.keras.mixed_precision.set_global_policy('float32') #make sure all weights and biased are in 'float32'

input_shape = (64, 64, 3)

model = Sequential([
    Input(shape=input_shape),
    Rescaling(1./255.0),             # bake in normalization
    Conv2D(8, 3, activation='relu', padding='valid'),
    MaxPooling2D(2),
    Conv2D(16, 3, activation='relu', padding='valid'),
    MaxPooling2D(2),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')    # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the model in Keras format
model.save(os.path.join(save_dir,'mobilenet_finetuned_binary_plant_village.keras'))

## Saving the Model to Google Drive

This section demonstrates how to save the fine-tuned MobileNetV2 model to a folder named `TinyPlants` in your Google Drive. By following these steps, you can ensure that the trained model is stored securely and can be accessed later for deployment or further experimentation.

### Steps:
1. **Mount Google Drive**: Connect your Google Drive to the Colab environment to enable file saving and retrieval.
2. **Create Folder**: Check if the `TinyPlants` folder exists in your Google Drive. If not, create it automatically.
3. **Save the Model**: Save the fine-tuned model in the `.keras` format to the `TinyPlants` folder for easy organization and access.


In [ ]:
import os
# Save the model in Keras format to the specified directory
# Define Google Drive save paths in the TinyPlants folder
save_dir = '/content/drive/My Drive/TinyPlants'

# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, 'binary_plant_village.keras')
model.save(model_path)

print(f"Model saved to {model_path}")


## Converting the Model to TensorFlow Lite with Integer Quantization

This section demonstrates how to convert the trained Keras model into a TensorFlow Lite model with integer quantization, ensuring that the input tensors have the correct dimensions required for TensorFlow Lite. Integer quantization optimizes the model for deployment on resource-constrained devices, like microcontrollers, while maintaining accuracy.

### Key Steps:

1. **Representative Dataset**:
   - A subset of the training data is used to calibrate the quantization process, ensuring accurate scaling of weights and activations during integer quantization.

2. **Correct Input Shape for Representative Dataset**:
   - TensorFlow Lite requires inputs to have a batch dimension, making the input shape `[batch_size, height, width, channels]`.
   - This issue is addressed by using `tf.expand_dims()` to add a batch dimension to the images in the representative dataset.

3. **Conversion Process**:
   - The TensorFlow Lite converter is configured to use integer quantization (`TFLITE_BUILTINS_INT8`) and fallback to TensorFlow operations (`SELECT_TF_OPS`) for unsupported layers.

4. **Save the Quantized Model**:
   - The quantized model is saved as `mobilenet_quantized_binary_plant_village.tflite` for deployment.

### Benefits of Integer Quantization:
- Reduces model size and memory usage, making it suitable for TinyML applications.
- Increases inference speed on devices with hardware acceleration for integer arithmetic.
- Maintains high accuracy through proper calibration using the representative dataset.

By the end of this section, the CNN model is converted into a lightweight, quantized format, ready for deployment on edge devices.



## Why we need a Representative Dataset
When you quantize a model, you’re reducing the numerical precision of:
- **Weights** (e.g., from `float32` → `int8`)
- **Activations / intermediate feature maps** (e.g., `float32` → `int8`)

To do that properly, TensorFlow Lite needs to figure out:
- The **range of values** that each tensor can take during inference (`min` and `max`)
- How to map those ranges into an 8-bit integer range (`-128` to `127` for signed int8)


## How the representative dataset is used
- The representative dataset is passed through the model **once during conversion** (not training).
- TFLite observes the **distribution of activations** at each layer.
- It then chooses an appropriate **scale factor** and **zero point** for each tensor.
- This calibration ensures that the model’s integer arithmetic closely approximates the original floating-point behavior.

**Without a representative dataset:**
- Tensor ranges might be guessed poorly (e.g., all layers get the same generic range).
- Quantization error could blow up, leading to a **huge drop in accuracy**.


## How good does it need to be?
- It **doesn’t need to be large**—usually 100–500 samples is enough.
- It **should reflect the real-world input distribution** your model will see.



In [ ]:
import tensorflow as tf

# Load the .keras model
#model = tf.keras.models.load_model(os.path.join(save_dir,'binary_plant_village.keras'))

# Define the representative dataset for quantization
def representative_dataset():
    for image, _ in train_dataset.unbatch().take(100):  # Take 100 samples from training data
        image = tf.expand_dims(image, axis=0)  # Add a batch dimension
        yield [tf.cast(image, tf.float32)]

# Convert the loaded Keras model to TensorFlow Lite format with integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.experimental_new_converter = True
# Enable fallback to allow unsupported ops to use float
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8,  # Use integer ops when possible
    tf.lite.OpsSet.TFLITE_BUILTINS,          # allow float    # Fallback to TensorFlow ops if needed
]

converter.inference_input_type = tf.int8  # Quantize input to int8
#converter.inference_output_type = tf.int8  # Quantize output to int8

# Convert the model
quantized_model = converter.convert()

# Save the quantized model
quantized_model_path = os.path.join(save_dir,'quantized_binary_plant_village.tflite')
with open(quantized_model_path, 'wb') as f:
    f.write(quantized_model)

print(f"Quantized model saved as '{quantized_model_path}'")




# Inspect TFLite Ops: Is the Model Fully Deployable (No TF Fallback)?

**What this cell does**
- **Loads a `.tflite` model** from disk into a `tf.lite.Interpreter`.
- **Lists all operators** used in the model via `interpreter._get_ops_details()`.
- **Flags TensorFlow fallback ops** (`SELECT_TF_OPS` / Flex).  
  - If any fallback ops are present, the model is **not** purely TFLite-builtins and **won’t** run on runtimes that require only builtins (e.g., TFLite Micro on many microcontrollers).
  - If **no** fallback ops are present, all ops are native TFLite builtins—**good for embedded deployment**.

**Why this matters**
- **Full TFLite builtin coverage** is required for microcontroller targets (TFLite Micro).  
- Models that rely on `SELECT_TF_OPS` need the larger TF runtime and are typically unsuitable for TinyML deployments.

**Printed output**
- A numbered list of ops like:
  - `Op 0: CONV_2D (TFLite Builtin)`
  - `Op 1: SELECT_TF_OP_* (Fallback to TensorFlow)`
- A final message stating whether the model includes TF fallback ops.

**Notes & limitations**
- This check focuses on **operator support**, not tensor **data types**:
  - To verify **full INT8 quantization**, also inspect:
    - `interpreter.get_input_details()` / `get_output_details()` to ensure `dtype == int8` and valid `(scale, zero_point)`.
    - `interpreter.get_tensor_details()` to confirm intermediates are quantized (look for quantization params).
- `interpreter._get_ops_details()` is a **private** API and may change across TF versions. Alternatives:
  - `tf.lite.experimental.Analyzer.analyze_model()` for a human-readable graph report.
  - Parse the FlatBuffer with the TFLite schema if you need a stable programmatic approach.

**Bottom line**
- **No `SELECT_TF_OPS` found** → the model is **builtins-only** and generally **deployable on microcontrollers**.  
- **Any `SELECT_TF_OPS` present** → not fully TFLite-native; **full INT8 TinyML deployment is unlikely** without modifying the model.


In [ ]:
import tensorflow as tf
import flatbuffers
import os

from tensorflow.lite.python import interpreter as interpreter_wrapper
from tensorflow.lite.python.util import get_tensor_name
from tensorflow.lite.python.schema_py_generated import OperatorCode, BuiltinOperator

def inspect_tflite_model(tflite_model_path):
    # Load the TFLite model
    with open(tflite_model_path, 'rb') as f:
        model_data = f.read()

    # Load model using the flatbuffer interpreter
    interpreter = tf.lite.Interpreter(model_content=model_data)
    interpreter.allocate_tensors()

    # Extract details
    details = interpreter.get_tensor_details()

    print("=== Operator List in TFLite Model ===")
    ops = interpreter._get_ops_details()

    fallback_ops = set()

    for i, op in enumerate(ops):
        op_name = op['op_name']
        if op_name.startswith('SELECT_TF_OP'):
            fallback_ops.add(op_name)
            print(f" Op {i}: {op_name} (Fallback to TensorFlow)")
        else:
            print(f"Op {i}: {op_name} (TFLite Builtin)")

    if fallback_ops:
        print("\nModel includes TensorFlow fallback ops. This means full int8 quantization may not be achievable.")
    else:
        print("\n All operators are natively supported by TFLite. Good for deployment on microcontrollers!")

# inpsec the saved .tflite file
inspect_tflite_model(os.path.join(save_dir, 'quantized_binary_plant_village.tflite'))


# Compare Original Keras vs Quantized TFLite (Size · Latency · Accuracy)

**What this cell does**
- **Loads models:**
  - Original Keras model (`.keras`)
  - Quantized TFLite model (`.tflite`)
- **Builds a TFLite evaluator:**
  - Creates an interpreter, feeds each test image, runs inference, and computes accuracy.
- **Measures latency:**
  - Uses wall-clock time to compute **seconds per sample** for both models.
- **Checks model size:**
  - Reports file sizes (KB) of the `.keras` and `.tflite` models.
- **Prints a side-by-side comparison** of size, latency, and accuracy.

**How accuracy is computed**
- **Keras:** `model.evaluate(test_dataset)` returns accuracy directly.
- **TFLite:** Iterates over `test_dataset.unbatch()`, runs `interpreter.invoke()`, applies `sigmoid` + `round` to get a binary prediction, and compares with labels.

**How latency is measured**
- Records start/end time around each evaluation loop and divides by the number of test samples to get **seconds/sample**.

**Outputs you’ll see**
1. **Model Size (KB)** for Keras vs TFLite  
2. **Latency (s/sample)** for both models  
3. **Accuracy** on the test set for both models

**Notes**
- If the TFLite model’s **input type is int8**, you typically need to quantize inputs using the interpreter’s input **scale** and **zero-point** from `input_details[0]['quantization']` rather than a raw cast to `int8`.  
- Ensure test preprocessing (resize/normalize) matches training; mismatches can skew accuracy and latency.


In [ ]:
import tensorflow as tf
import time
import os


# Load the original Keras model
original_model = tf.keras.models.load_model(os.path.join(save_dir,'binary_plant_village.keras'))

# Load the TFLite quantized model
tflite_model_path = (os.path.join(save_dir,'quantized_binary_plant_village.tflite'))
with open(tflite_model_path, 'rb') as f:
    tflite_model = f.read()

# Helper function to evaluate the TFLite model
def evaluate_tflite_model(tflite_model, test_dataset):
    # Initialize the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    correct_predictions = 0
    total_samples = 0
    for image, label in test_dataset.unbatch():
        # Preprocess the image
        input_data = tf.cast(image, tf.int8 if input_details[0]['dtype'] == tf.int8 else tf.float32)
        input_data = tf.expand_dims(input_data, axis=0)  # Add batch dimension

        # Set the input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data.numpy())
        interpreter.invoke()

        # Get the prediction
        output = interpreter.get_tensor(output_details[0]['index'])
        predicted_label = tf.round(tf.sigmoid(output)).numpy()[0][0]
        if predicted_label == label.numpy():
            correct_predictions += 1
        total_samples += 1

    # Calculate accuracy
    return correct_predictions / total_samples

# Evaluate the original model
original_start_time = time.time()
original_test_loss, original_test_acc = original_model.evaluate(test_dataset, verbose=0)
original_latency = (time.time() - original_start_time) / len(list(test_dataset.unbatch()))
original_size = os.path.getsize((os.path.join(save_dir,'mobilenet_finetuned_binary_plant_village.keras'))) / 1024  # in KB

# Evaluate the TFLite model
tflite_start_time = time.time()
tflite_test_acc = evaluate_tflite_model(tflite_model, test_dataset)
tflite_latency = (time.time() - tflite_start_time) / len(list(test_dataset.unbatch()))
tflite_size = os.path.getsize(tflite_model_path) / 1024  # in KB

# Print the comparison results
print(f"Model Comparison:")
print(f"1. Model Size:")
print(f"   - Original Model: {original_size:.2f} KB")
print(f"   - TFLite Model: {tflite_size:.2f} KB")
print(f"2. Latency (seconds per sample):")
print(f"   - Original Model: {original_latency:.4f} s/sample")
print(f"   - TFLite Model: {tflite_latency:.4f} s/sample")
print(f"3. Accuracy on Test Data:")
print(f"   - Original Model: {original_test_acc:.4f}")
print(f"   - TFLite Model: {tflite_test_acc:.4f}")


# Save Sample Images & Export Quantized Int8 Arrays (TinyML-ready)

### 1) Quantize images to Int8 using the TFLite model’s calibration
- **Reads input quantization params** from the `.tflite`:
  - `scale` (float), `zero_point` (int) via `interpreter.get_input_details()[0]["quantization"]`.
- **Selects one sample per class** (`healthy`, `unhealthy`) directly from the dataset (already in **[0,1]**), resizes to **64×64**, and **quantizes to int8**:
  - **Quantization formula:**  
    `q = round(x / scale + zero_point)`  
    then clip to `[-128, 127]` for `int8`.
- **Exports a C header** `samples_one_per_class.h` with two arrays:
  - `const int8_t healthy_64x64x3[12288]`
  - `const int8_t unhealthy_64x64x3[12288]`
- **Layout:** NHWC (row-major). `12288 = 64 × 64 × 3` elements (RGB).

## Why quantization here (not a raw cast)?
- A plain cast (`float32 → int8`) would distort values (most would become 0).
- Using the model’s **(scale, zero_point)** ensures the integer inputs match what the model was **calibrated** to expect during post-training quantization.

**Dequantize (what TFLite does internally):**  
`real ≈ scale × (q − zero_point)`

**Quantize (what we do for input):**  
`q = round(real / scale + zero_point)`


## Outputs you’ll see
1. **Saved JPEGs** under `healthy/` and `unhealthy/`.
2. Console printouts of the model’s **input dtype** and **(scale, zero_point)**.
3. A header file: **`samples_one_per_class.h`** containing two `int8_t` arrays ready for Arduino/TFLite Micro.


## Notes & tips
- Keep preprocessing identical to training (e.g., if you used `/255.0`, do that before quantization).
- The arrays are **signed int8**; if your model expects `uint8`, adjust accordingly.



In [ ]:
import os, numpy as np, tensorflow as tf
import matplotlib.pyplot as plt

# === Inputs you already have ===
# - train_dataset: a tf.data.Dataset yielding (image, label) with labels {0: healthy, 1: unhealthy}
# - tflite_model_path: path to your quantized .tflite

TARGET_SIZE = (64, 64)  # adjust if your model expects a different size
tflite_model_path = os.path.join(save_dir, "quantized_binary_plant_village.tflite")

# 1) Read input quantization (scale, zero_point) from the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
in_scale, in_zero = input_details["quantization"]  # (scale, zero_point)

print("TFLite input quantization:", (in_scale, in_zero))
print("Expected input dtype:", input_details["dtype"], "shape:", input_details["shape"])

# 2) Choose the preprocessing that matches your *float* model input before quantization
#    If your model expects [0,1] floats (common when you used Rescaling(1/255) or manual /255):
def preprocess_float(x):
    # If your dataset already yields [0,1] floats, leave as-is.
    # If dataset is uint8 [0,255], uncomment the next line:
    # x = x / 255.0
    return x

# 3) Pick 2 samples from each class (0 = healthy, 1 = unhealthy)
def collect_two_per_class(dataset):
    got = {0: [], 1: []}
    for img, lab in dataset.unbatch():
        lab = int(lab.numpy())
        if len(got[lab]) < 2:
            # Resize to target size
            img = tf.image.resize(img, TARGET_SIZE, method="bilinear")
            got[lab].append(img.numpy())
        if len(got[0]) == 2 and len(got[1]) == 2:
            break
    if len(got[0]) < 2 or len(got[1]) < 2:
        raise ValueError("Not enough samples found for one of the classes.")
    return got

samples = collect_two_per_class(train_dataset)  # {0: [img0,img1], 1: [img0,img1]}

# 4) Quantize to int8 using scale/zero-point (NHWC)
def float_to_int8(nhwc_float, scale, zero):
    # nhwc_float should be the float *input domain* your model expects (often [0,1])
    q = np.round(nhwc_float / scale + zero)
    q = np.clip(q, -128, 127).astype(np.int8)
    return q

# 5) Show images + print arrays; also accumulate for a .h file
def show_and_prepare(arr, title):
    plt.figure()
    # For visualization, clip to [0,1] if you used that range; otherwise normalize for display
    vis = preprocess_float(arr.copy())
    vis = np.clip(vis, 0.0, 1.0)
    plt.imshow(vis.astype(np.float32))
    plt.title(title)
    plt.axis("off")

arrays_c = []  # (name, int8_array_flat, shape)
names = []

for cls, name_prefix in [(0, "healthy"), (1, "unhealthy")]:
    for i, img in enumerate(samples[cls]):
        f32 = preprocess_float(img.astype(np.float32))            # HxWxC float
        q8  = float_to_int8(f32, in_scale, in_zero)               # HxWxC int8
        show_and_prepare(f32, f"{name_prefix} #{i} (display)")
        flat = q8.flatten()
        var_name = f"{name_prefix}_{i}_64x64x3_int8"
        arrays_c.append((var_name, flat, q8.shape))
        names.append(var_name)

plt.show()

# 6) Print full arrays (int8) to the notebook
for var_name, flat, shape in arrays_c:
    print(f"\n// {var_name} shape {shape}, length {flat.size}")
    # Print as comma-separated values
    # (Comment out if too long)
    print("{")
    # chunk printing for readability
    for s in range(0, flat.size, 32):
        line = ", ".join(str(int(v)) for v in flat[s:s+32])
        print("  " + line + ("," if s + 32 < flat.size else ""))
    print("}")

# 7) Write a C header you can include in Arduino projects
header_path = os.path.join(save_dir, "samples_int8.h")
with open(header_path, "w") as f:
    f.write("#pragma once\n#include <stdint.h>\n\n")
    f.write("// Quantization: x_int8 = round(x_float/scale + zero_point)\n")
    f.write(f"// scale = {in_scale:.8g}, zero_point = {int(in_zero)}\n")
    f.write(f"// layout = NHWC, size = {TARGET_SIZE[0]}x{TARGET_SIZE[1]}x3\n\n")
    for var_name, flat, shape in arrays_c:
        f.write(f"// {var_name} shape {shape}\n")
        f.write(f"const int8_t {var_name}[{flat.size}] = {{\n")
        for s in range(0, flat.size, 32):
            line = ", ".join(str(int(v)) for v in flat[s:s+32])
            f.write("  " + line + ("," if s + 32 < flat.size else "") + "\n")
        f.write("};\n\n")
print("Saved header:", header_path)

# (Optional) Also save each array as a .npy if you want to load later:
# for var_name, flat, shape in arrays_c:
#     np.save(os.path.join(save_dir, f"{var_name}.npy"), flat.reshape(shape))


In [ ]:
import os, numpy as np, tensorflow as tf

TARGET_SIZE = (64, 64)
tflite_model_path = os.path.join(save_dir, "quantized_binary_plant_village.tflite")

# Read input quantization
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
inp = interpreter.get_input_details()[0]
in_scale, in_zero = inp["quantization"]          # (scale, zero_point)
in_dtype = inp["dtype"]                          # np.int8 / np.uint8 / np.float32

if in_dtype == np.int8:
    clip_lo, clip_hi = -128, 127
    c_type = "int8_t"
elif in_dtype == np.uint8:
    clip_lo, clip_hi = 0, 255
    c_type = "uint8_t"
else:
    raise ValueError("Model input is float32; no quantization needed. (Change model or skip header.)")

def float_to_q(nhwc_float):
    # nhwc_float is already in [0,1] as you said
    q = np.round(nhwc_float / in_scale + in_zero)
    return np.clip(q, clip_lo, clip_hi).astype(in_dtype)

def collect_one_per_class(dataset):
    got = {0: None, 1: None}  # 0=healthy, 1=unhealthy
    for img, lab in dataset.unbatch():
        lab = int(lab.numpy())
        if got[lab] is None:
            # resize to model input
            img = tf.image.resize(img, TARGET_SIZE, method="bilinear")
            got[lab] = img.numpy().astype(np.float32)  # already [0,1]
            if got[0] is not None and got[1] is not None:
                break
    if got[0] is None or got[1] is None:
        raise ValueError("Couldn't find one sample per class.")
    return got[0], got[1]

healthy_f32, unhealthy_f32 = collect_one_per_class(train_dataset)
healthy_q = float_to_q(healthy_f32)       # (64,64,3)
unhealthy_q = float_to_q(unhealthy_f32)

# Write a header with exactly two arrays
header_path = os.path.join(save_dir, "samples_one_per_class.h")
with open(header_path, "w") as f:
    f.write("#pragma once\n#include <stdint.h>\n\n")
    f.write("// Input quantization: q = round(x/scale + zero_point)\n")
    f.write(f"// scale = {in_scale:.8g}, zero_point = {int(in_zero)}, dtype = {c_type}\n")
    f.write(f"// layout = NHWC, size = {TARGET_SIZE[0]}x{TARGET_SIZE[1]}x3\n\n")
    for name, arr in [("healthy_64x64x3", healthy_q), ("unhealthy_64x64x3", unhealthy_q)]:
        flat = arr.flatten()
        f.write(f"const {c_type} {name}[{flat.size}] = {{\n")
        for s in range(0, flat.size, 32):
            line = ", ".join(str(int(v)) for v in flat[s:s+32])
            f.write("  " + line + ("," if s + 32 < flat.size else "") + "\n")
        f.write("};\n\n")

print("Saved header:", header_path)


# Convert the `.tflite` file into a C/C++ Array for Firmware

## Why do this?
On many microcontrollers you can’t read files from a filesystem. Converting your TensorFlow Lite model (`.tflite`) into a C/C++ byte array lets you **compile the model into your firmware** so it lives in program memory (flash) and can be passed directly to TFLite Micro.

.cc file wil contain something like this
unsigned char quantized_binary_plant_village_tflite[] = { 0x1c, 0x00, 0x00, ... };
unsigned int quantized_binary_plant_village_tflite_len = 123456;


In [ ]:
!xxd -i /content/drive/My\ Drive/TinyPlants/quantized_binary_plant_village.tflite > model.cc
